## Demo Run with Indicator Coordinates

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%cd /Users/Leonard/Desktop/NN_imp/SA_classifier

In [ ]:
from SA_geometry import *

In [ ]:
#This coordinate serves as an indicator for nearby SA
coords = (51.520404, -0.071797)

### Build a 2D Model of the area

In [ ]:
#Create the surrounding bbox (see source)
bbox = create_bbox(coords, polygonize=True)
#Get data and model the polygon objects within the bbox
polygons = extract_polygons(
    create_bbox(coords, polygonize=False)
)
#Aggregate the polygons into one big MultiPolygon object (in order to plot it)
aggregated = aggregate_polygons(polygons, bbox)

### Extract relevant facades

In [ ]:
plot(aggregated, "blue"), plot(coords, "purple", size=6.5)

In [ ]:
#Extract the *target polygon* which contains the facades (see source)
target = extract_target_polygon(bbox, polygons)

In [ ]:
plot(bbox, "black"), plot(aggregated, "blue"), plot(coords, "green", size=10)

In [ ]:
#Extract the *facades* from the target polygon (see source)
facades = extract_facades(list(target.boundary.difference(bbox.boundary))) 
#Segment the facades (if needed)
facades = segment(facades, size=10)

In [ ]:
plot(bbox, "black"), plot(aggregated, "blue"), plot(facades, "orange"), plot(coords, "green", size=7.33)

## Get the image data for all extracted facades

In [ ]:
# If a nearby gsv_point exists, retrieve the photo of the facade
# Adjust parameters accordingly.(see source)
data = get_gsv_data(facades=facades, target=target,
                    photogeometric_fov=False,
                    as_dataframe=True)
data.to_csv("demo_run.csv")

### Submit the data to the trained Neural Network

In [ ]:
from SA_predict import classify
import pandas as pd
from shapely.wkt import loads as to_geo
from IPython.display import Image, display

In [ ]:
#load data
data = pd.read_csv("demo_run.csv", index_col=0)

In [ ]:
#load data
data = pd.read_csv("demo_run.csv", index_col=0)
#classify data
data = classify(data, to_csv=True)

### Evaluation

In [ ]:
from IPython.display import Image, display, HTML

In [ ]:
SA_candidates = [ to_geo(facade) for facade in list(data[data["pred"] > .99]["facade"]) ]

In [ ]:
plot(aggregated, "blue"), plot(facades, "orange"), plot(coords, "green", size=7.33), plot(SA_candidates, "red")

In [ ]:
for gsv in list(data[data["pred"] == 1]["photo"]):
    display(Image(url=gsv, width=300, height=300)); display(HTML('<h9>$P(SA) = 1$</h3>'))